# Lab 2.4.6: Uploading Models to Hugging Face Hub

**Module:** 2.4 - Hugging Face Ecosystem  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐ (Intermediate)

---

## Learning Objectives

By the end of this notebook, you will:
- [ ] Create a Hugging Face account and authentication token
- [ ] Write a professional model card
- [ ] Upload a fine-tuned model to the Hub
- [ ] Organize model repositories properly
- [ ] Share models with the community
- [ ] Update and version your models

---

## Prerequisites

- Completed: Labs 2.4.4 and 2.4.5 (Fine-tuning with Trainer and LoRA)
- Have: A **Hugging Face account** (free at [huggingface.co/join](https://huggingface.co/join)) - **CREATE BEFORE STARTING**
- Have: A **Write access token** from [huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)
- Have: A fine-tuned model to upload (or we'll create one in this notebook)

---

## Real-World Context

You've trained an amazing model. Now what?

**Sharing your models enables:**
- Reproducibility: Others can verify and build on your work
- Collaboration: Team members can access the same model
- Deployment: Use the Hub's inference API
- Recognition: Build your ML portfolio and reputation

**Success stories:**
- Researchers share models alongside papers for reproducibility
- Companies share specialized models for domain problems
- Individuals build portfolios that lead to job opportunities
- Open source models drive the entire AI community forward!

---

## ELI5: What is the Hugging Face Hub?

> **Imagine the Hub is like YouTube, but for AI models.**
>
> Just like YouTube:
> - You create an account
> - You upload your "content" (models instead of videos)
> - You write a description (model card instead of video description)
> - Others can view, download, and use your work
> - Popular models get more attention and usage
>
> **Why share?**
> - Your model might help someone solve their problem
> - The community can improve your work
> - You build a portfolio of your ML skills
> - Companies look at your Hub profile when hiring!
>
> **The Model Card** is like the "About" section of your video:
> - What does the model do?
> - How was it trained?
> - What are its limitations?
> - How should people use it?

---

## Part 1: Authentication Setup

In [ ]:
# Install required packages
# Note: These packages are pre-installed in the NGC PyTorch container.
# Running pip install ensures you have compatible versions.
# If NOT using NGC container, ensure you have ARM64-compatible packages for DGX Spark.

!pip install -q "transformers>=4.35.0" "huggingface_hub>=0.19.0" "datasets>=2.14.0" "peft>=0.6.0" "evaluate>=0.4.0"

import torch
import numpy as np
import os
from huggingface_hub import HfApi, login, whoami, create_repo
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import evaluate
import warnings
warnings.filterwarnings('ignore')

print("Packages loaded!")

### Creating Your Access Token

To upload to the Hub, you need an access token:

1. Go to [huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)
2. Click "New token"
3. Give it a name (e.g., "dgx-spark-training")
4. Select "Write" access (needed for uploads)
5. Copy the token

In [ ]:
# Method 1: Interactive login (recommended for first time)
# This will prompt for your token and save it
# login()

# Method 2: Login from terminal (run this in a cell if needed)
# !huggingface-cli login

# Method 3: Environment variable (for scripts/automation)
# os.environ['HF_TOKEN'] = 'your_token_here'

print("Authentication options:")
print("1. Run: login() - Interactive login")
print("2. Run: !huggingface-cli login - Terminal login")
print("3. Set HF_TOKEN environment variable")

In [ ]:
# Check if logged in
try:
    user_info = whoami()
    print(f"Logged in as: {user_info['name']}")
    print(f"Organizations: {user_info.get('orgs', [])}")
    logged_in = True
except Exception as e:
    print(f"Not logged in: {e}")
    print("\nPlease run login() or !huggingface-cli login first")
    logged_in = False

---

## Part 2: Preparing Your Model

Let's create a simple fine-tuned model to upload.

In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import Trainer, TrainingArguments
import numpy as np
import evaluate

# Load and prepare data
print("Loading IMDB dataset...")
imdb = load_dataset("imdb")

# Use small subset for quick training
small_train = imdb['train'].shuffle(seed=42).select(range(2000))
small_val = imdb['train'].shuffle(seed=42).select(range(2000, 2500))
small_test = imdb['test'].shuffle(seed=42).select(range(500))

dataset = DatasetDict({
    'train': small_train,
    'validation': small_val,
    'test': small_test
})

print(f"Train: {len(dataset['train'])}")
print(f"Validation: {len(dataset['validation'])}")
print(f"Test: {len(dataset['test'])}")

In [ ]:
# Load model and tokenizer
model_name = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1}
)

print(f"Model: {model.config.model_type}")
print(f"Parameters: {model.num_parameters():,}")

In [ ]:
# Tokenize
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=256
    )

tokenized = dataset.map(tokenize_function, batched=True, remove_columns=['text'])
tokenized = tokenized.rename_column('label', 'labels')

print(f"Tokenized columns: {tokenized['train'].column_names}")

In [ ]:
# Train
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="./model_for_upload",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    bf16=True,
    logging_steps=50,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['validation'],
    compute_metrics=compute_metrics,
)

print("Training...")
trainer.train()
print("Training complete!")

In [ ]:
# Evaluate
print("\nEvaluating on test set...")
test_results = trainer.evaluate(tokenized['test'])

print(f"\nTest Results:")
print(f"  Accuracy: {test_results['eval_accuracy']:.4f}")
print(f"  Loss: {test_results['eval_loss']:.4f}")

# Store for model card
model_accuracy = test_results['eval_accuracy']

---

## Part 3: Creating a Model Card

A model card is ESSENTIAL. It tells users:
- What the model does
- How to use it
- What data it was trained on
- Known limitations and biases

In [ ]:
# Create a comprehensive model card
# NOTE: Replace YOUR_USERNAME with your actual Hugging Face username before uploading!

model_card_content = f"""
---
language:
- en
license: mit
tags:
- sentiment-analysis
- text-classification
- distilbert
- imdb
datasets:
- imdb
metrics:
- accuracy
model-index:
- name: distilbert-imdb-sentiment
  results:
  - task:
      type: text-classification
      name: Sentiment Analysis
    dataset:
      type: imdb
      name: IMDB Movie Reviews
    metrics:
    - type: accuracy
      value: {model_accuracy:.4f}
---

# DistilBERT fine-tuned for IMDB Sentiment Analysis

This model is a fine-tuned version of [distilbert-base-uncased](https://huggingface.co/distilbert-base-uncased) 
on the IMDB movie reviews dataset for sentiment analysis.

## Model Description

- **Base Model:** distilbert-base-uncased
- **Task:** Binary sentiment classification (positive/negative)
- **Language:** English
- **Training Data:** IMDB Movie Reviews (subset)

## Training Details

### Hardware
- **Platform:** NVIDIA DGX Spark
- **GPU:** NVIDIA Blackwell GB10 Superchip
- **Memory:** 128GB Unified LPDDR5X

### Training Hyperparameters
- Learning rate: 2e-5
- Batch size: 16
- Epochs: 2
- Warmup ratio: 0.1
- Max sequence length: 256
- Precision: bfloat16

### Training Data
- Training samples: 2,000
- Validation samples: 500
- Test samples: 500

## Evaluation Results

| Metric | Value |
|--------|-------|
| Accuracy | {model_accuracy:.2%} |

## Usage

### Using the Pipeline API (Recommended)

```python
from transformers import pipeline

# Replace YOUR_USERNAME with your actual username!
classifier = pipeline("sentiment-analysis", model="YOUR_USERNAME/distilbert-imdb-sentiment")

result = classifier("This movie was amazing!")
# Output: [{{'label': 'POSITIVE', 'score': 0.99}}]
```

### Using AutoModel

```python
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Replace YOUR_USERNAME with your actual username!
tokenizer = AutoTokenizer.from_pretrained("YOUR_USERNAME/distilbert-imdb-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("YOUR_USERNAME/distilbert-imdb-sentiment")

inputs = tokenizer("This movie was terrible!", return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

predictions = torch.softmax(outputs.logits, dim=1)
# predictions[0][0] = negative probability
# predictions[0][1] = positive probability
```

## Limitations

- **Domain:** Trained on movie reviews; may not generalize well to other domains
- **Length:** Best with reviews under 256 tokens
- **Language:** English only
- **Bias:** May reflect biases present in IMDB reviews

## Intended Use

This model is intended for:
- Classifying sentiment of movie/entertainment reviews
- Educational purposes (learning about fine-tuning)
- Baseline for sentiment analysis tasks

Not intended for:
- High-stakes decision making
- Medical or legal sentiment analysis
- Languages other than English

## Citation

If you use this model, please cite:

```bibtex
@misc{{distilbert-imdb-sentiment,
  author = {{DGX Spark Student}},
  title = {{DistilBERT fine-tuned for IMDB Sentiment Analysis}},
  year = {{2025}},
  publisher = {{Hugging Face}},
  howpublished = {{\\url{{https://huggingface.co/YOUR_USERNAME/distilbert-imdb-sentiment}}}}
}}
```

## Training Curriculum

This model was trained as part of the **DGX Spark AI Curriculum**, 
Module 9: Hugging Face Ecosystem.

---

*Last updated: 2025*
"""

# Save to file
with open("./model_for_upload/README.md", "w") as f:
    f.write(model_card_content)

print("Model card created!")
print("\nPreview (first 50 lines):")
print("="*50)
for line in model_card_content.split("\n")[:50]:
    print(line)

### Model Card Best Practices

| Section | Why It Matters |
|---------|----------------|
| **Description** | Quick understanding of what the model does |
| **Training Details** | Reproducibility |
| **Evaluation** | Trust and comparison |
| **Usage Examples** | Easy adoption |
| **Limitations** | Responsible AI - prevents misuse |
| **Citation** | Credit for your work |

---

## Part 4: Uploading to the Hub

In [ ]:
# Save model and tokenizer locally first
save_path = "./model_for_upload"

print("Saving model and tokenizer...")
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

# List saved files
print("\nSaved files:")
for f in os.listdir(save_path):
    size_mb = os.path.getsize(os.path.join(save_path, f)) / 1e6
    print(f"  {f}: {size_mb:.2f} MB")

In [ ]:
# Method 1: Using push_to_hub() (simplest)
# This uploads everything including the model card!

# Define your repository name
# Replace 'your-username' with your actual username!
repo_name = "distilbert-imdb-sentiment-demo"

print("To upload your model, run:")
print(f"""\n
# Option 1: Using trainer (includes training info)
trainer.push_to_hub("{repo_name}")

# Option 2: Using model and tokenizer directly
model.push_to_hub("{repo_name}")
tokenizer.push_to_hub("{repo_name}")

# Option 3: Upload entire directory
from huggingface_hub import upload_folder
upload_folder(
    folder_path="{save_path}",
    repo_id="your-username/{repo_name}",
    repo_type="model"
)
""")

In [ ]:
# Uncomment to actually upload (requires login)
# trainer.push_to_hub(repo_name)
# print(f"\nModel uploaded to: https://huggingface.co/YOUR_USERNAME/{repo_name}")

### Method 2: Using the Hub API (More Control)

In [ ]:
from huggingface_hub import HfApi, upload_file, upload_folder

api = HfApi()

# Create repository first (optional, push_to_hub creates it)
# api.create_repo(repo_id=repo_name, repo_type="model", private=False)

print("Hub API options:")
print("""
# Create a new repository
api.create_repo(repo_id="model-name", repo_type="model", private=False)

# Upload a single file
api.upload_file(
    path_or_fileobj="./path/to/file",
    path_in_repo="filename",
    repo_id="username/repo-name",
    repo_type="model"
)

# Upload entire folder
api.upload_folder(
    folder_path="./local/folder",
    repo_id="username/repo-name",
    repo_type="model"
)

# Delete repository
api.delete_repo(repo_id="username/repo-name", repo_type="model")
""")

---

## Part 5: Uploading LoRA Adapters

LoRA adapters can also be uploaded!

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

# Create a LoRA model
base_model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_lin", "v_lin"],
    lora_dropout=0.1,
    task_type=TaskType.SEQ_CLS,
    modules_to_save=["classifier"]
)

lora_model = get_peft_model(base_model, lora_config)
lora_model.print_trainable_parameters()

In [ ]:
# Save LoRA adapter
lora_save_path = "./lora_adapter_for_upload"
lora_model.save_pretrained(lora_save_path)

# Add model card for adapter
# NOTE: Replace YOUR_USERNAME with your actual Hugging Face username!
adapter_card = """
---
library_name: peft
base_model: distilbert-base-uncased
---

# LoRA Adapter for DistilBERT Sentiment Analysis

This is a LoRA adapter for sentiment analysis.

## Usage

```python
from transformers import AutoModelForSequenceClassification
from peft import PeftModel

# Load base model
base = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

# Load adapter (replace YOUR_USERNAME with your actual username!)
model = PeftModel.from_pretrained(base, "YOUR_USERNAME/lora-adapter-name")
```

## Configuration

- Rank (r): 8
- Alpha: 16
- Target modules: q_lin, v_lin
"""

with open(os.path.join(lora_save_path, "README.md"), "w") as f:
    f.write(adapter_card)

print("LoRA adapter saved!")
print(f"\nTo upload: lora_model.push_to_hub('your-lora-adapter-name')")

---

## Part 6: Managing Your Repository

In [ ]:
# Repository management examples
print("Repository Management Commands:")
print("""
# List your models
from huggingface_hub import list_models
my_models = list(list_models(author="your-username"))

# Get model info
info = api.model_info("username/model-name")
print(f"Downloads: {info.downloads}")
print(f"Likes: {info.likes}")

# Update model card
api.upload_file(
    path_or_fileobj="./new_README.md",
    path_in_repo="README.md",
    repo_id="username/model-name"
)

# Make private/public
api.update_repo_visibility("username/model-name", private=True)

# Add collaborators (for organizations)
# Manage through web UI or API
""")

In [ ]:
# Using Git for version control
print("Using Git with Hugging Face Hub:")
print("""
# Clone your model repository
git clone https://huggingface.co/username/model-name
cd model-name

# Make changes
# ... edit files ...

# Commit and push
git add .
git commit -m "Update model card"
git push

# Use Git LFS for large files (automatic for model files)
git lfs install
git lfs track "*.bin"
git lfs track "*.safetensors"
""")

---

## Part 7: Best Practices for Model Sharing

In [ ]:
print("""
MODEL SHARING BEST PRACTICES
============================

1. NAMING CONVENTIONS
   - Use descriptive names: 'bert-base-sentiment' not 'my_model_1'
   - Include base model: 'distilbert-imdb-classifier'
   - Include task/domain: 'roberta-medical-ner'

2. MODEL CARD ESSENTIALS
   - Clear description of what the model does
   - Training data and methodology
   - Evaluation metrics and results
   - Code examples for usage
   - Known limitations and biases
   - License information

3. FILE ORGANIZATION
   - README.md (model card)
   - config.json (model configuration)
   - model.safetensors (weights - prefer safetensors over pytorch_model.bin)
   - tokenizer files
   - training_args.json (optional)

4. VERSIONING
   - Use Git tags for major versions
   - Document changes in model card
   - Keep backward compatibility when possible

5. LICENSING
   - Choose appropriate license (MIT, Apache 2.0, etc.)
   - Check base model license compatibility
   - Consider data licensing implications

6. REPRODUCIBILITY
   - Include training script or link to it
   - Document random seeds used
   - List package versions
   - Provide training logs if possible
""")

---

## Try It Yourself: Upload Your Model

Complete the following steps to upload your own model:

1. Login to Hugging Face
2. Fine-tune a model (or use one from previous tasks)
3. Create a comprehensive model card
4. Upload to the Hub
5. Test loading your model from the Hub

<details>
<summary>Hint</summary>

```python
# Step 1: Login
from huggingface_hub import login
login()

# Step 2: Have your model ready (from Task 9.4 or 9.5)

# Step 3: Create model card (copy template above)

# Step 4: Upload
trainer.push_to_hub("your-model-name")

# Step 5: Test
from transformers import pipeline
pipe = pipeline("text-classification", model="your-username/your-model-name")
print(pipe("Test sentence"))
```
</details>

In [ ]:
# YOUR CODE HERE
# Upload your model to the Hub!




---

## Common Mistakes

### Mistake 1: Forgetting the Model Card

In [ ]:
# WRONG: Upload without model card
# model.push_to_hub("my-model")  # No README.md!

# CORRECT: Create model card first
# 1. Save model card to output directory
# 2. Then push

print("Always include a model card!")
print("Users need to know how to use your model.")

### Mistake 2: Wrong Token Permissions

In [ ]:
# WRONG: Using read-only token for upload
# Error: "You don't have permission to push to this repo"

# CORRECT: Use a token with WRITE access
# Create at: huggingface.co/settings/tokens
# Select "Write" when creating the token

print("Token types:")
print("  Read: Can only download")
print("  Write: Can upload and modify")

### Mistake 3: Not Including All Required Files

In [ ]:
# WRONG: Only uploading model weights
# model.save_pretrained("./model")
# api.upload_file("./model/pytorch_model.bin", ...)
# Missing: config.json, tokenizer files!

# CORRECT: Upload entire directory or use push_to_hub
# model.push_to_hub("model-name")
# tokenizer.push_to_hub("model-name")

print("Required files for loading a model:")
print("  - config.json")
print("  - model.safetensors or pytorch_model.bin")
print("  - tokenizer_config.json")
print("  - vocab.txt or tokenizer.json")
print("  - special_tokens_map.json")

---

## Checkpoint

You've learned:
- ✅ How to authenticate with the Hugging Face Hub
- ✅ How to create professional model cards
- ✅ How to upload models and adapters
- ✅ How to manage your model repositories
- ✅ Best practices for sharing models

---

## Challenge: Complete Model Sharing Workflow

Complete end-to-end workflow:
1. Fine-tune a model for a unique task
2. Create a comprehensive model card
3. Upload to Hub
4. Test inference API
5. Share the link!

In [ ]:
# YOUR CHALLENGE CODE HERE




---

## Further Reading

- [Model Cards Guide](https://huggingface.co/docs/hub/model-cards)
- [Hub Python Library](https://huggingface.co/docs/huggingface_hub)
- [Sharing Best Practices](https://huggingface.co/docs/hub/repositories-getting-started)
- [Model Card Template](https://github.com/huggingface/huggingface_hub/blob/main/src/huggingface_hub/templates/modelcard_template.md)

---

## Cleanup

In [ ]:
import shutil
import gc

# Clean up local files
for path in ["./model_for_upload", "./lora_adapter_for_upload"]:
    if os.path.exists(path):
        shutil.rmtree(path)
        print(f"Removed {path}")

gc.collect()
torch.cuda.empty_cache()

print("\nCleanup complete!")

---

## Module 2.4 Complete!

Congratulations! You've completed the entire Hugging Face Ecosystem module!

**What you've learned:**
1. ✅ Lab 2.4.1: Hub Exploration - Finding and evaluating models
2. ✅ Lab 2.4.2: Pipeline Showcase - Quick inference with pipelines
3. ✅ Lab 2.4.3: Dataset Processing - Efficient data handling
4. ✅ Lab 2.4.4: Trainer Fine-tuning - Professional training workflows
5. ✅ Lab 2.4.5: LoRA Introduction - Parameter-efficient fine-tuning
6. ✅ Lab 2.4.6: Model Upload - Sharing with the community

**You can now:**
- Find and use any model from the 500K+ on the Hub
- Process datasets of any size efficiently
- Fine-tune models with professional best practices
- Use LoRA to train large models on limited hardware
- Share your work with the global ML community

**Next:** Continue your journey with Domain 3: LLM Systems!

Great job! 🎉